In [77]:
import requests
import ipaddress
import pandas as pd
import numpy as np 
from concurrent.futures import ThreadPoolExecutor, as_completed
from anticaptchaofficial.recaptchav2proxyless import *
import time
import os 


csbp = "https://noafiliacion.csbp.com.bo/"
cns = "https://www.cns.gob.bo/Servicios/NoAfiliacion"
cps = "https://afiliado.cps.org.bo/"

download_directory = "/home/mavino/Descargas"
# Generando los elementos 
from selenium import webdriver

profile = webdriver.FirefoxProfile()

profile.set_preference("browser.download.dir", download_directory)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip,application/pdf,document/pdf")

profile.set_preference("browser.link.open_newwindow", 3 )
profile.set_preference("browser.link.open_newwindow.restriction", 0 )

 

In [81]:

navegador = webdriver.Firefox(profile)

# Recolección de datos 

# CSBP
driver = navegador.get(csbp)
window_csbp = navegador.window_handles[0]



navegador.execute_script("window.open('about:blank', '_blank');")
window_cns = navegador.window_handles[1]
navegador.switch_to.window(window_cns)
navegador.get(cns)


navegador.execute_script("window.open('about:blank', '_blank');")
window_cps = navegador.window_handles[2]
navegador.switch_to.window(window_cps)
navegador.get(cps)



In [84]:
boton = navegador.find_by_Xpath("/html/body/main/div/form/button")

AttributeError: 'WebDriver' object has no attribute 'find_by_Xpath'

In [79]:
navegador.window_handles

WebDriverException: Message: Failed to decode response from marionette


In [5]:
fecha_i = '20210803'

fecha_dt = pd.to_datetime(fecha_i, format= "%Y%m%d")

fecha_dt.strftime("%d/%m/%Y")

DF_PENDIENTES = pd.read_csv("{}.csv".format(fecha_i))
DF_PENDIENTES["cubiculo"] = 0
DF_PENDIENTES["atencion" ] = fecha_dt.strftime("%d/%m/%Y")
DF_PENDIENTES["oficina"] = "1"
#DF_PENDIENTES["sexo"] = ""
#DF_PENDIENTES["estado_civil"] = ""
DF_PENDIENTES["direccion"] = ""
DF_PENDIENTES["email"] = ""
DF_PENDIENTES["tipo_atencion"] = "normal"
# DF_PENDIENTES["extension"] = "" # Añadir el condicional de verificar si el campo está vacío 
DF_PENDIENTES["telefono"] = ""
DF_PENDIENTES = DF_PENDIENTES.fillna("")
DF_PENDIENTES.drop("contacto",axis=1,inplace=True)
#DF_PENDIENTES.drop("extencion",axis=1,inplace=True)
DF_PENDIENTES.shape

(7, 17)

In [6]:
DF_PENDIENTES.head()

,ci,extension,emision,ap_paterno,ap_materno,nombre,fecha_nac,celular,estado_civil,sexo,cubiculo,atencion,oficina,direccion,email,tipo_atencion,telefono
0,13496856,,La Paz,IBAÑEZ,YUJRA,VERONICA,1999-12-13,77789116,Soltero,Femenino,0,03/08/2021,1,,,normal,
1,9081754,,La Paz,CLAROS,SOLARES,JAVIER,2000-12-04,69823087,Soltero,Masculino,0,03/08/2021,1,,,normal,
2,6005381,,La Paz,QUISPE,CHOQUE,IRMA,1976-07-01,69724755,Casado,Femenino,0,03/08/2021,1,,,normal,
3,8278278,,La Paz,CUTILE,BALBOA,RITA,1989-05-22,68009216,Soltero,Femenino,0,03/08/2021,1,,,normal,
4,669682,,Oruro,ALCOCER,ANGULO,NESTOR,1954-09-07,77298904,Casado,Masculino,0,03/08/2021,1,,,normal,


# Ejecución en paralelo

In [7]:
DATOS = [data_i for data_i in DF_PENDIENTES.to_dict(orient="records") ]

In [10]:
DATOS = DATOS[:-1]

In [11]:
TOTAL_RESPUESTAS = []
URL = []

Si todo funciona correctamente hasta aquí, procedemos con la consulta masiva
# CONSULTA PARALELA MASIVA

In [12]:
start = time.time()
processes = []
with ThreadPoolExecutor(max_workers=25) as executor:
    RESULTADOS_FINALES = {executor.submit(ObtenerFichaREC, cli_i) : cli_i for cli_i in DATOS}
    
    for ficha_i in as_completed(RESULTADOS_FINALES):
            url = RESULTADOS_FINALES[ficha_i]
            try:
                data = ficha_i.result()
            except Exception as exc:
                print('{} EXCEPTION {}'.format(url["ci"], exc))
            else:
                print('{} RESULTADO {} '.format(url["ci"], data))    

print(f'Total Ejecución FICHAS : {time.time() - start}')

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

Time taken captcha_i: 48.15830898284912
Time taken captcha_i: 50.321168661117554


/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Reintentos restantes 19 {'ci': 8278278, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'CUTILE', 'ap_materno': 'BALBOA', 'nombre': 'RITA', 'fecha_nac': '1989-05-22', 'celular': 68009216, 'estado_civil': 'Soltero', 'sexo': 'Femenino ', 'cubiculo': 0, 'atencion': '03/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '161.56.49.122'}, 'cookie': '6OvO0zGbMAS6rNMxL1qFbj1AX5oSIL', 'recaptcha': '03AGdBq24Edx7DPzykLcxwtHAPXRzWx7IaGqKupCfelqRpLWDbJGCY3hVNfE3zEw_wFXXyGKiJd0uUKKz4MgpGrocXkoUM_SBUkAUTo4B2tnp-evyFXouBC7bRA_hxyD0sIG-fK121tOV0I1QeWZA8dMjTKB7n_tu1QtwRf4cTQbYux7BAQhH6sF7__nItfJyuf5FYW_3_zsre-Ube_fWMwXuLWGCSwhoPCqPxPwsBpc-yD1BVg4rQq38eVh3ADY9_BtRk11OXx5wH3CbgtXzY0xAxtCFajF9ipBLj2cLGsY-GF9cymQ7r0d6OmiO7YsqTKhwCNnsj7PkyMXMVBBgDybXVidXa89I-FIViXliQlR7MLlqTUGBh5fHun_lQ16HoeQ2zBoCPsWu1Mf8NMMYY4MFyjpw5TN1qQ2QdR3hF0YZRuPhqz_HKIWjhfXDmGjcYZksw3BCgnyPwB7Hk6s2ztW2IMhaiEFiX6QG8PlvtySEfj9-gEjKtU764fk1lAj4JUbYu3tyeueAnWpr-tI1XyGLzlim

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 6&persona=Nestor Alcocer Angulo&oficina=Regional La Paz&codigo=UjAr2r
Nestor Alcocer Angulo 08:36 

669682 RESULTADO ({'ci': 669682, 'extension': '', 'emision': 'Oruro', 'ap_paterno': 'ALCOCER', 'ap_materno': 'ANGULO', 'nombre': 'NESTOR', 'fecha_nac': '1954-09-07', 'celular': 77298904, 'estado_civil': 'Casado', 'sexo': 'Masculino', 'cubiculo': 0, 'atencion': '03/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '181.227.246.55'}, 'cookie': 'OgGYLAjqXMj2dKMa0wf2EnpmknVNpM', 'recaptcha': '03AGdBq26xIQTsy4Kbi2pFaC-0p9AwphJYZcrvmPbrqevUjUzislI1xdBveEbupR_eTXNlPmzrWIaNn5nrbQdyZHun2q34X3gxeNGgRMTRFBgOt8o6OWgKhQFd8bTXEw936X9gX7fC6JqVIidB9XDx3DSM0pPIBgxntvGy5v5GF57kwJE55KHTnY3ToxuVTgYmF_MTEZGTIhjtfUFshtcSH5hJTjujfPiQfjQEACPvWxvvUWTilXoF97vQT1uemB-UashTgL1_YFKqFP2HP7PCPrXTU0k2L8_dd8DWzBkux9julRfvO-ENbZywI5tn_uwm-AZ

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 5&persona=ERIKA MACHICADO QUISPE&oficina=Regional La Paz&codigo=df5gmb
ERIKA MACHICADO QUISPE 08:36 

4816303 RESULTADO ({'ci': 4816303, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'MACHICADO', 'ap_materno': 'QUISPE', 'nombre': 'ERIKA', 'fecha_nac': '1980-07-27', 'celular': 73501937, 'estado_civil': 'Soltero', 'sexo': 'Femenino ', 'cubiculo': 0, 'atencion': '03/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '161.138.121.193'}, 'cookie': 'ukj7z7jVnWxrnuuZltvEmnq01lrG9F', 'recaptcha': '03AGdBq24LdzxIWdPyVSDSSbk0Js00fypRtOJwOjXXSM2Lqg88cIApFI8kFzuSHteB2ALb49k_QxfxFMEXqYwztusoHq5diMQl0r4JwlTBqdi9dH4bu4wgvm8I_CFdKFZk7Mzyvd2LjWZYlZpNd-wZ1jH7m6l1esBh6B6Jmy6dcaAOWig8R5pzPNdFizu6IihdIpBazifA2EEGAxqy0uvVPFWrxGWzf6yC8WyS57nhouTiIZ34ihtQHlzV91m4Dfhyl6thG-LLPaXpsYIr8D9APqu__GO9032gmUIIpJDd45Lo9t1YK5Fx9B_BuiR

/home/mavino/anaconda3/envs/SEVIMAworkflow/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cnslp.gob.bo'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 7&persona=IRMA QUISPE CHOQUE&oficina=Regional La Paz&codigo=Qo4Gxi
IRMA QUISPE CHOQUE 08:36 

6005381 RESULTADO ({'ci': 6005381, 'extension': '', 'emision': 'La Paz', 'ap_paterno': 'QUISPE', 'ap_materno': 'CHOQUE', 'nombre': 'IRMA', 'fecha_nac': '1976-07-01', 'celular': 69724755, 'estado_civil': 'Casado', 'sexo': 'Femenino ', 'cubiculo': 0, 'atencion': '03/08/2021', 'oficina': '1', 'direccion': '', 'email': '', 'tipo_atencion': 'normal', 'telefono': '', 'ip': {'ip': '161.22.188.155'}, 'cookie': '93GO6RC181y69qTLLfuvnHca6RWYMN', 'recaptcha': '03AGdBq24AeN_YfY8WbQxyv-Zsax2aRAfYAgwC8pRSzX5tQXWgoA60sv5DyEAMIrtdQrk-jm0wlLWyaMfgRQlpfgELhq_Ge9DXDmKF5TY3qpqOSsadAprxe0WizwLFc0AjEQYeWVMQb82S9zA9qfgYW4usHBpYoOg5vxdkeiP8mXC7ifYCzj15Z6xwUPXXqawMna51YnH6e40IPyBDQLW5BmHMILkxHPBK6g6PUkKh6YP-R3hlj48_QfA3x9Wvmrm4dCg1SIvlOlsFgG_gSaJ2sxdTOyn161s8H8xsMkvcCw038p44mlaMLLiWrnD7Zqsgh0xK08U-c

In [13]:
ronda_i = [ A for A in as_completed(RESULTADOS_FINALES)]

In [14]:
ronda_i

[<Future at 0x7f742c02ffd0 state=finished returned tuple>,
 <Future at 0x7f741c194128 state=finished returned tuple>,
 <Future at 0x7f742c02f6a0 state=finished returned tuple>,
 <Future at 0x7f741c194470 state=finished returned tuple>,
 <Future at 0x7f741c194630 state=finished returned tuple>,
 <Future at 0x7f742c02fc18 state=finished returned tuple>]

In [15]:
[i.result()[1] for i in ronda_i]

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [16]:
URLS = []

In [17]:
for i in ronda_i:
    if i.result()[1].ok:
        salida = i.result()[1].json()["respuesta"]

        oficina_i = oficinasDF.loc[ oficinasDF.id == salida["oficina"], "nombre"].values[0]

        url_i = "http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha={nro_ficha}&fecha_atencion={fecha_atencion}&hora_aproximada={hora_aproximada}&cubiculo={cubiculo}&persona={persona}&oficina={oficina}&codigo={codigo}".format(
            nro_ficha     = salida["numero_atencion"]
            ,fecha_atencion   = pd.to_datetime(salida["fecha"]).strftime("%d/%m/%Y")
            ,hora_aproximada    = salida["hora_aproximada"]
            ,cubiculo  = salida["cubiculo"]
            ,persona  = salida["persona"]
            ,oficina = oficina_i 
            ,codigo = salida["codigo"])
        print(url_i)
        URLS.append(url_i)

http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 4&persona=RITA CUTILE BALBOA&oficina=Regional La Paz&codigo=6IyRyo
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 5&persona=VERONICA IBAÑEZ YUJRA&oficina=Regional La Paz&codigo=7tx04Y
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 7&persona=IRMA QUISPE CHOQUE&oficina=Regional La Paz&codigo=Qo4Gxi
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 6&persona=Nestor Alcocer Angulo&oficina=Regional La Paz&codigo=UjAr2r
http://127.0.0.1:8000/fichasCNS/generarFicha?nro_ficha=3&fecha_atencion=03/08/2021&hora_aproximada=08:36&cubiculo=Modulo 3&persona=Javier Sergio Claros Solares&oficina=Regional La Paz&codigo=2HBWLd
http://127.0.0.1:8000/fichasCNS/generarFicha

In [18]:
URLS.sort()
len(URLS)

6

# Obtener PDF basado en el QR 

In [19]:
import os, shutil

In [20]:

for filename in os.listdir(download_directory):
    file_path = os.path.join(download_directory, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [21]:
for i in URLS:
    navegador.get(i)
    time.sleep(1)

In [22]:
len(URLS)

6

In [43]:
# Renombramiento automático

In [44]:
requests.get("http://127.0.0.1:8000/fichasCNS/rename")

<Response [200]>

In [45]:
try: 
    os.rename(  os.path.join(download_directory, "TOTAL.pdf") , 
                os.path.join(download_directory, "TOTAL_{fec}.pdf".format(fec = fecha_i)) )
except : 
    print("Algo malio sal ...")

In [46]:
navegador.close()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=56141): Max retries exceeded with url: /session/2ff721fa-f5d8-4087-a620-ac3358fb2bf5/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f741c616f60>: Failed to establish a new connection: [Errno 111] Connection refused',))

# GENERACION DEL ARCHIVO DE LINKS 

In [47]:
fecha_i

'20210803'

In [48]:
DF_PENDIENTES = pd.read_csv("{}.csv".format(fecha_i))

# B = open("E:\\sevima\\SEVIMAworkflow-master\\fichasCNS\\FICHAS\\"+  "LinksMensajes_{}.html".format(fecha_i),"w") # Windows 
B = open("/media/F/sevima/SEVIMAworkflow-master/fichasCNS/FICHAS/"+  "LinksMensajes_{}.html".format(fecha_i),"w")
for data_i in DF_PENDIENTES.to_dict(orient="records") :
    # print("https://wa.me/591{}?text=Buen+dia%2C+adjunta+su+ficha.+Le+recordamos+estar+15+minutos+antes+de+su+hora+de+atenci%C3%B3n+para+no+perder+su+turno.".format(data_i["celular"]))
#    B.writelines('<h1> <a href="https://wa.me/591{}?text=Muy%20buenas%20noches%2C%20lamento%20comunicarle%20que%20el%20sistema%20de%20la%20Caja%20Nacional%20de%20Salud%20parece%20estar%20en%20mantenimiento%20por%20lo%20tanto%20no%20se%20pueden%20sacar%20fichas.%20Esta%20eventualidad%20escapa%20de%20mis%20manos%20para%20poder%20brindarle%20el%20servicio.%20Espero%20que%20ma%C3%B1ana%20se%20habilite%20otra%20vez%20el%20sistema%20y%20en%20funci%C3%B3n%20a%20eso%20yo%20le%20comunico%20las%20novedades%2C%20otra%20vez%20mil%20disculpas.%20"  target="_blank" > {} {} {} {} </a> </h1> <br>'.format(data_i["celular"],data_i["celular"], data_i["nombre"], data_i["ap_paterno"], data_i["contacto"] ))
    B.writelines('<h1> <a href="https://wa.me/591{}?text=Buenas+tardes%2C+adjunta+su+ficha.+Le+recordamos+estar+15+minutos+antes+de+su+hora+de+atenci%C3%B3n+para+no+perder+su+turno."  target="_blank" > {} {} {} {} </a> </h1> <br>'.format(data_i["celular"],data_i["celular"], data_i["nombre"], data_i["ap_paterno"], data_i["contacto"] ))
print("Se genero el archivo exitosamente! ")
B.close()

Se genero el archivo exitosamente! 


In [49]:
# Generando Archivo de compresión
os.system("cd {down_dir} && rar a FICHAS_{fec}.rar *".format(fec = fecha_i, down_dir = download_directory))

0

# Guardando las respuetas 

In [50]:
import pickle
pickle_file = open('respuestas{}.pickle'.format(fecha_i), 'wb')
pickle.dump([i.result() for i in ronda_i], pickle_file)
pickle_file.close()